# k-Nearest Neighbours test

In [1]:
import networkx as nx 
import random
import matplotlib.pyplot as plt
from sklearn import neighbors,model_selection
import numpy as np
import csv

In [2]:
# Attributes : Age; Blood_Pressure; BMI; Plasma_level; Autoimmune_Disease; Adverse_events; Drug_in_serum; Liver_function; Activity_test; Secondary_test
Attributes = ['Age', 'Blood_Pressure', 'BMI', 'Plasma_level', 'Autoimmune_Disease', 'Adverse_events', 'Drug_in_serum', 'Liver_function', 'Activity_test', 'Secondary_test']

# We create a list of list to contain the data
# Each data[i] represents the data for the attribute i
#X_train = np.array([])
data = []
with open ('autoimmune.txt', 'r') as f:
    for row in csv.reader(f,delimiter='\t'):
        data.append(row)

# The fifth row is the one of Autoimmune_Disease = data[4]
# And Autoimmune_Disease is the attribute we want to predict so :
X_train = np.zeros((len(data[0]),len(data)-1))
for patient in range (len(data[0])):
    patient_data = []
    for attribute in range (len(data)):
        if attribute != 4:
            patient_data.append(float(data[attribute][patient]))
    X_train[patient] = patient_data
Y_train = []   
for x in data[4]:
    if x=='positive':
        Y_train.append(1)
    else: Y_train.append(0)
Y_train = np.array(Y_train)

In [3]:
def euclidian_dist(a,b):
    dist = []
    if type(a)=='int':
        return sqrt((a-b)**2)
    else:
        for i in range(len(a)):
            dist.append(((a[i]-b[i])**2)**0.5)
        return dist

In [4]:
def kNN(X_train, Y_train, X_test, k, a=0.5):
    
    dist = []
    for i in range(len(X_train)):
        dist.append(euclidian_dist(X_train[i],X_test))
    
    votes = [x for _,x in sorted(zip(dist,Y_train), reverse = False)]
    
    classification = 0
    for j in range(k):
        classification += votes[j]
        
    if classification/k > a:
        return np.array(1)
    else: return np.array(0)

In [5]:
# The fifth row is the one of Autoimmune_Disease = data[4]
# And Autoimmune_Disease is the attribute we want to predict so :
X_train = [[]*(len(data)-1)]*len(data[0])
for patient in range (len(data[0])):
    patient_data = []
    for attribute in range (len(data)):
        if attribute != 4:
            patient_data.append(float(data[attribute][patient]))
    X_train[patient] = patient_data
Y_train = []   
for x in data[4]:
    if x=='positive':
        Y_train.append(1)
    else: Y_train.append(0)

# 10-folder CV
Xparts = []
Yparts = []

start = 0
end = int(len(X_train)/10)
for part in range(10):  # We create the 10 folders
    if end>=len(X_train):
        Xparts.append(X_train[start:])
        Yparts.append(Y_train[start:])
    else:
        Xparts.append(X_train[start:end])
        Yparts.append(Y_train[start:end])
    start = end
    end = start + int(len(X_train)/10)

In [6]:
scores = []
for part in range(10):
    accuracy = 0
    x_test = Xparts[part]  
    y_test = Yparts[part]
    x_train,y_train = [],[]
    for j in range(10):
        if part!=j:
            x_train += Xparts[j]
            y_train += Yparts[j]
    
    for i in range(len(x_test)):
        if y_test[i]==kNN(x_train, y_train, x_test[i], 8):
            accuracy +=1
    scores.append(accuracy/len(x_test))
scores = np.array(scores)
print(scores.mean())

0.7027027027027026


In [7]:
k=8
KNN = neighbors.KNeighborsClassifier(n_neighbors=k)
scores = []
for part in range(10):
    accuracy = 0
    x_test = Xparts[part]  
    y_test = Yparts[part]
    x_train,y_train = [],[]
    for j in range(10):
        if part!=j:
            x_train += Xparts[j]
            y_train += Yparts[j]
    
    for i in range(len(x_test)):
        KNNfit = KNN.fit(np.array(x_train), np.array(y_train))
        if y_test[i]==KNNfit.predict(np.array(x_test[i]).reshape(1, -1)):
            accuracy +=1
    scores.append(accuracy/len(x_test))
scores = np.array(scores)
print(scores.mean())

0.7621621621621621
